In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from time import time
%matplotlib inline

In [2]:
df = pd.read_csv('data/clean/train_clean_v2.csv')
df_targets = pd.read_csv('data/clean/train_labels.csv')

In [3]:
df.head()

,fecha_dato,fecha_dato_year,fecha_dato_month,fecha_dato_day,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,...,indresi,indext,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento
0,2015-01-28,2015,1,28,1375586,1,1,1,35,2015-01-12,...,2,1,20,1,1.0,29.0,7,1.0,87218.10,1
1,2015-01-28,2015,1,28,1050611,1,1,1,23,2012-08-10,...,2,2,1,1,1.0,13.0,23,0.0,35548.74,2
2,2015-01-28,2015,1,28,1050612,1,1,1,23,2012-08-10,...,2,1,1,1,1.0,13.0,23,0.0,122179.11,2
3,2015-01-28,2015,1,28,1050613,1,1,1,22,2012-08-10,...,2,1,7,1,1.0,50.0,6,0.0,119775.54,2
4,2015-01-28,2015,1,28,1050615,1,1,1,23,2012-08-10,...,2,1,1,1,1.0,45.0,16,0.0,22220.04,2


In [4]:
df_copy = df.copy()

En el dataframe de copia elimino las columnas de fechas y el código de cada usuario, dejando sólo la información necesaria para el entrenamiento

In [5]:
x = df_copy.drop(['fecha_dato', 'fecha_alta', 'ncodpers'], axis=1).as_matrix()
y = df_targets.as_matrix()
print('Tamaño de datasets', x.shape, y.shape)

((10795392, 24), (10795392, 24))

### Entrenamiento con un solo target (ind_ahor_fin_ult1) - posición 0
Este entrenamiento se hace con los datos del mismo mes

In [6]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

El valor 10094948 se calculó en notebooks pasados, éste indica la fila en la que inicia el último mes de registro

In [21]:
x_train = x[:10094948]
x_test = x[10094948:]
y_train = y[:10094948, 0]
y_test = y[10094948:, 0]
print('Train', x_train.shape, y_train.shape)
print('Test', x_test.shape, y_test.shape)

Train (10094948, 24) (10094948,)
Test (700444, 24) (700444,)


In [8]:
%%time
rf.fit(x_train, y_train)

CPU times: user 2min 23s, sys: 1.67 s, total: 2min 25s
Wall time: 2min 25s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [9]:
rf.score(x_test, y_test)

0.99997858501179249

### Balanceo de ceros y unos
Revisión de cantidad de unos y ceros en cada columna del target

In [10]:
for i, col in enumerate(df_targets.columns):
    print(i, col, np.unique(df_targets[col].as_matrix(), return_counts=True))

0 ind_ahor_fin_ult1 (array([0, 1]), array([10794143,     1249]))
1 ind_aval_fin_ult1 (array([0, 1]), array([10795098,      294]))
2 ind_cco_fin_ult1 (array([0, 1]), array([3711072, 7084320]))
3 ind_cder_fin_ult1 (array([0, 1]), array([10790848,     4544]))
4 ind_cno_fin_ult1 (array([0, 1]), array([9842499,  952893]))
5 ind_ctju_fin_ult1 (array([0, 1]), array([10683904,   111488]))
6 ind_ctma_fin_ult1 (array([0, 1]), array([10695598,    99794]))
7 ind_ctop_fin_ult1 (array([0, 1]), array([9300781, 1494611]))
8 ind_ctpp_fin_ult1 (array([0, 1]), array([10286697,   508695]))
9 ind_deco_fin_ult1 (array([0, 1]), array([10777424,    17968]))
10 ind_deme_fin_ult1 (array([0, 1]), array([10776266,    19126]))
11 ind_dela_fin_ult1 (array([0, 1]), array([10289841,   505551]))
12 ind_ecue_fin_ult1 (array([0, 1]), array([9823961,  971431]))
13 ind_fond_fin_ult1 (array([0, 1]), array([10579747,   215645]))
14 ind_hip_fin_ult1 (array([0, 1]), array([10724700,    70692]))
15 ind_plan_fin_ult1 (array([0,

Prueba de balanceo con una sola columna

In [36]:
col = 2
y_one_index = df_targets.iloc[y[:10094948, col] == 1, col].index #Índices de las filas con unos
y_zero_index = df_targets.iloc[y[:10094948, col] == 0, col].index #Índices de las filas con ceros
n = 100000 #Cantidad de la muestra
n = np.min(np.unique(df_targets.iloc[:, col].as_matrix(), return_counts=True)[1])
print(n)

#Lista de unos y ceros
sample = np.sort(list(np.random.choice(y_one_index, size=n)) + list(np.random.choice(y_zero_index, size=n)))

3711072


In [41]:
targets = df_targets.columns

In [47]:
np.unique(df_targets[targets[2]].as_matrix(), return_counts=True)[1]

array([3711072, 7084320])

In [37]:
limit = 10094948
x2_train = x[y_sample]
y2_train = y[y_sample, col]
x2_test = x[limit:]
y2_test = y[limit:, col]

In [38]:
print("Train", x2_train.shape, y2_train.shape)
print("Test", x2_test.shape, y2_test.shape)

Train (7422144, 24) (7422144,)
Test (700444, 24) (700444,)


In [39]:
%%time
rf2 = RandomForestClassifier()
rf2.fit(x2_train, y2_train)

CPU times: user 4min 7s, sys: 3.86 s, total: 4min 11s
Wall time: 4min 11s


In [40]:
rf2.score(x2_test, y2_test)

0.87607574624095574